In [6]:
import keyring
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=OPENAI_API_KEY)

template = "You are an assistant that helps users find information about movies."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Find information about the movie {movie_title}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

response = chat(chat_prompt.format_prompt(movie_title="Inception").to_messages())

print(response.content)

c:\Users\woojin\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\woojin\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


"Inception" is a 2010 science fiction action film directed by Christopher Nolan. The movie stars Leonardo DiCaprio as a professional thief who steals information by entering the subconscious minds of his targets through their dreams. The film explores the concept of dream sharing and features a complex narrative that blurs the lines between reality and dreams. "Inception" received critical acclaim for its originality, visual effects, and performances. It was also a commercial success, grossing over $800 million worldwide.


# Summarization Chain Example

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

# initialize language model
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=OPENAI_API_KEY)

# Load the summarization chain
summarize_chain = load_summarize_chain(llm)

# load the document using PyPDFLoader
document_loader = PyPDFLoader(file_path='./data/The One Page Linux Manual.pdf')
document = document_loader.load()

# summarize the document
summary = summarize_chain(document)
print(summary['output_text'])

c:\Users\woojin\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


"The One Page Linux Manual provides a summary of useful Linux commands for starting and stopping processes, accessing and mounting file systems, finding files and text within files, using the X Window System, moving, copying, deleting, and viewing files, installing software, user administration, tips and tricks, configuration files, file permissions, X shortcuts, and printing. It also includes information on how to start and stop the print daemon, display print queue jobs, remove jobs from the queue, and print files."


# QA Chain Example

In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

prompt = PromptTemplate(template="Question: {question}\nAnswer:", input_variables=["question"])

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=OPENAI_API_KEY)
chain = LLMChain(llm=llm, prompt=prompt)

chain.run("what is the meaning of life?")

c:\Users\woojin\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\woojin\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


'The meaning of life is a deeply philosophical question that has been debated for centuries. Different people and cultures have different beliefs about the purpose and meaning of life. Some believe that the meaning of life is to seek happiness and fulfillment, others believe it is to serve a higher power or fulfill a specific destiny, while others believe that life has no inherent meaning and it is up to each individual to create their own meaning. Ultimately, the meaning of life is a personal and subjective question that each person must grapple with and find their own answer to.'

# Building a New Articles Summarizer

In [1]:
import keyring
import requests
from newspaper import Article

heades = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"

session = requests.Session()

try:
    response = session.get(article_urls, headers=heades, timeout=10)
    
    if response.status_code == 200:
        article = Article(article_urls)
        article.download()
        article.parse()
        
        print(f"Title: {article.title}")
        print(f"Text: {article.text}")
        
    else:
        print(f"Failed to fetch article at {article_urls}")

except Exception as e:
    print(f"Error occurred while fetching article at {article_urls}: {e}")

Title: Meta claims its new AI supercomputer will set records
Text: Ryan Daws is a senior editor at TechForge Media with over a decade of experience in crafting compelling narratives and making complex topics accessible. His articles and interviews with industry leaders have earned him recognition as a key influencer by organisations like Onalytica. Under his leadership, publications have been praised by analyst firms such as Forrester for their excellence and performance. Connect with him on X (@gadget_ry) or Mastodon (@gadgetry@techhub.social)

Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and 

In [2]:
from langchain.schema import (
    HumanMessage
)

# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==========
Title: {article_title}

{article_text}
==========

Write a summary of the previous article.
"""

prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]

In [7]:
from langchain.chat_models import ChatOpenAI
import keyring
import os

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
# load the model
chat = ChatOpenAI(model_name='gpt-4-turbo', temperature=0, openai_api_key=OPENAI_API_KEY)

# generate summary
summary = chat(messages)
print(summary.content)

Meta (formerly Facebook) has announced the development of the AI Research SuperCluster (RSC), an AI supercomputer that it claims will be the world's fastest upon its completion in mid-2022. The RSC is designed to train large natural language processing (NLP) and computer vision models, with capabilities to handle models with trillions of parameters. Meta aims to use the RSC for advanced AI applications, such as real-time voice translations for large groups and enhancing the metaverse with AI-driven technologies. The RSC is expected to be significantly faster than Meta's current systems, improving training times and incorporating enhanced security and privacy features to utilize real-world data safely. This development marks a significant step in Meta's AI research, focusing on performance, reliability, security, and privacy at a large scale.


In [9]:
# prepare template for prompt
template = """You are an advanced AI assistant that summarizes online articles into bulleted lists.

Here's the article you need to summarize.

==========
Title: {article_title}

{article_text}
==========

Now provide a summarized version of the article in a bulleted list format.
"""

# format prompt
prompt = template.format(article_title=article.title, article_text=article.text)

# generate summary
summary = chat([HumanMessage(content=prompt)])
print(summary.content)


- **Meta's New AI Supercomputer**: Meta (formerly Facebook) has announced the development of the AI Research SuperCluster (RSC), which it claims will be the world's fastest AI supercomputer upon completion.
  
- **Current Status and Capabilities**: The RSC is not yet fully built but is expected to be completed by mid-2022. It is already being used by Meta's researchers to train large natural language processing (NLP) and computer vision models.

- **Performance Expectations**: Once fully operational, RSC is projected to be 20x faster than Meta’s current V100-based clusters, 9x faster at running the NVIDIA Collective Communication Library (NCCL), and 3x faster at training large-scale NLP workflows.

- **Training Efficiency**: A model with tens of billions of parameters can be trained in three weeks on RSC, a significant improvement from the nine weeks required with previous systems.

- **Applications and Goals**: Meta aims to use RSC to develop new AI systems for real-time voice transla

In [10]:
# prepare template for prompt
template = """You are an advanced AI assistant that summarized online articles into bulleted lists in French.
 

{article_text}
==================

Now, provide a summarized version of the article in a bulleted list format, in French.
"""

# format prompt 
prompt = template.format(article_title=article.title, article_text=article.text)

# generate summary
summary = chat([HumanMessage(content=prompt)])
print(summary.content)

- Meta (anciennement Facebook) a dévoilé un superordinateur IA, nommé AI Research SuperCluster (RSC), prétendu être le plus rapide au monde une fois achevé.
- Le RSC, qui n'est pas encore totalement construit, devrait être finalisé à mi-2022.
- Les chercheurs de Meta utilisent déjà le RSC pour entraîner des modèles de traitement du langage naturel (NLP) et de vision par ordinateur de grande envergure.
- Le but du RSC est de permettre la formation de modèles avec des trillions de paramètres.
- Meta envisage que le RSC puisse, par exemple, faciliter des traductions vocales en temps réel pour des groupes de personnes parlant différentes langues, afin de collaborer sur des projets de recherche ou de jouer à des jeux en réalité augmentée (AR).
- Le RSC devrait être 20 fois plus rapide que les clusters actuels de Meta basés sur V100, 9 fois plus rapide pour exécuter la bibliothèque de communication collective NVIDIA (NCCL) et 3 fois plus rapide pour l'entraînement de workflows NLP à grande é

# LlamaIndex

## LlamaHub Wikipedia Integration

In [11]:
import os
 
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['ACTIVE_LOOP'] = keyring.get_password('activeloop', 'key_for_windows')
WikipediaReader = download_loader('WikipediaReader')
loader = WikipediaReader()


C:\Users\woojin\AppData\Local\Temp\ipykernel_14192\1281470272.py:6: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  WikipediaReader = download_loader('WikipediaReader')


In [9]:
documents = loader.load_data(pages=['Natural Language Processing', 'Artificial Intelligence'])
len(documents)

2

## Save on DeepLake

In [16]:
from llama_index.core.vector_stores import DeepLakeVectorStore

ImportError: cannot import name 'DeepLakeVectorStore' from 'llama_index.core.vector_stores' (c:\Users\woojin\anaconda3\envs\llm\Lib\site-packages\llama_index\core\vector_stores\__init__.py)